## Import libraries


In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import models
import os
import shutil
import sys

## Dataset

In [4]:
labels = pd.read_csv('/content/labels.csv')
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [10]:
dataset_dir = '/content/drive/MyDrive/train'


In [11]:
from pathlib import Path
print(Path.cwd())

/content


## Number of Classes

In [5]:
n_class = len(labels.breed.unique())
n_class

120

## Pre-Processing

In [12]:
import os

def make_dir(x):
    if os.path.exists(x)==False:
        os.makedirs(x)

base_dir = './subset'
make_dir(base_dir)

In [13]:
train_dir = os.path.join(base_dir, 'train')
make_dir(train_dir)

In [14]:
breeds = labels.breed.unique()
for breed in breeds:
    # Make folder for each breed
    _ = os.path.join(train_dir, breed)
    make_dir(_)

    # Copy images to the corresponding folders
    images = labels[labels.breed == breed]['id']
    for image in images:
        source = os.path.join(dataset_dir, f'{image}.jpg')
        destination = os.path.join(train_dir, breed,f'{image}.jpg')
        shutil.copyfile(source, destination)

In [15]:
breeds

array(['boston_bull', 'dingo', 'pekinese', 'bluetick', 'golden_retriever',
       'bedlington_terrier', 'borzoi', 'basenji', 'scottish_deerhound',
       'shetland_sheepdog', 'walker_hound', 'maltese_dog',
       'norfolk_terrier', 'african_hunting_dog',
       'wire-haired_fox_terrier', 'redbone', 'lakeland_terrier', 'boxer',
       'doberman', 'otterhound', 'standard_schnauzer',
       'irish_water_spaniel', 'black-and-tan_coonhound', 'cairn',
       'affenpinscher', 'labrador_retriever', 'ibizan_hound',
       'english_setter', 'weimaraner', 'giant_schnauzer', 'groenendael',
       'dhole', 'toy_poodle', 'border_terrier', 'tibetan_terrier',
       'norwegian_elkhound', 'shih-tzu', 'irish_terrier', 'kuvasz',
       'german_shepherd', 'greater_swiss_mountain_dog', 'basset',
       'australian_terrier', 'schipperke', 'rhodesian_ridgeback',
       'irish_setter', 'appenzeller', 'bloodhound', 'samoyed',
       'miniature_schnauzer', 'brittany_spaniel', 'kelpie', 'papillon',
       'borde

In [16]:
batch_size = 128

In [18]:
datagen = ImageDataGenerator(rescale=1./255, # rescale pixel values to [0,1] to reduce memory usage
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split


train_generator = datagen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='sparse',
    subset='training')

validation_generator = datagen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='sparse',
    subset='validation')

Found 8221 images belonging to 120 classes.
Found 2001 images belonging to 120 classes.


## Models Used

In [19]:
from tensorflow.keras.applications import InceptionResNetV2

inception_bottleneck = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

219055592/219055592 [==============================] - 7s 0us/step


In [20]:
from tensorflow.keras.applications.vgg19 import VGG19

vgg_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

80134624/80134624 [==============================] - 1s 0us/step


In [21]:
from tensorflow.keras.applications import InceptionV3

InceptionV3_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


87910968/87910968 [==============================] - 1s 0us/step


In [22]:
InceptionV3_model.trainable = False

model = models.Sequential()
model.add(InceptionV3_model)
model.add(GlobalAveragePooling2D())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(n_class, activation='softmax'))


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#model.summary()

## Training the model

In [25]:
#early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)


history = model.fit(train_generator,
          #batch_size=batch_size,
          steps_per_epoch = train_generator.samples // batch_size,
          validation_data = validation_generator,
          validation_steps = validation_generator.samples // batch_size,
          epochs=5,
          verbose=1)
          #callbacks=[early_stop])

Epoch 1/5
64/64 [==============================] - 1299s 20s/step - loss: 1.1352 - accuracy: 0.6946 - val_loss: 0.8465 - val_accuracy: 0.7557
Epoch 2/5
64/64 [==============================] - 1270s 20s/step - loss: 0.7343 - accuracy: 0.7793 - val_loss: 0.8809 - val_accuracy: 0.7432
Epoch 3/5
64/64 [==============================] - 1251s 20s/step - loss: 0.6636 - accuracy: 0.7957 - val_loss: 0.8318 - val_accuracy: 0.7557
Epoch 4/5
64/64 [==============================] - 1251s 20s/step - loss: 0.5945 - accuracy: 0.8137 - val_loss: 0.8189 - val_accuracy: 0.7557
Epoch 5/5
64/64 [==============================] - 1260s 20s/step - loss: 0.5088 - accuracy: 0.8392 - val_loss: 0.8126 - val_accuracy: 0.7750


## Testing the model

In [27]:
!mkdir test_data
!cp -r "/content/drive/MyDrive/test" test_data

mkdir: cannot create directory ‘test_data’: File exists


In [28]:
datagen_test = ImageDataGenerator(rescale=1./255) # rescale pixel values to [0,1] to reduce memory usage

test_generator = datagen_test.flow_from_directory(
    directory="test_data",
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

Found 2025 images belonging to 1 classes.


In [30]:
images_names = os.listdir('/content/drive/MyDrive/test')
images_ids = [id.split('.')[0] for id in images_names]

In [31]:
probabilities = model.predict(test_generator)

16/16 [==============================] - 249s 15s/step


In [32]:
rows = []
for index, pred in enumerate(probabilities):
    pred_format = [format(z, '.4f') for z in pred]
    pred_format.insert(0, images_ids[index])
    rows.append(pred_format)

In [33]:
fields = breeds.tolist()
fields.insert(0, "id")

In [34]:
import csv
with open('output.csv', 'w') as f:

    # using csv.writer method from CSV package
    write = csv.writer(f)

    write.writerow(fields)
    write.writerows(rows)

In [3]:
output=pd.read_csv('/content/output.csv')
output.head()

,id,boston_bull,dingo,pekinese,bluetick,golden_retriever,bedlington_terrier,borzoi,basenji,scottish_deerhound,...,blenheim_spaniel,silky_terrier,sussex_spaniel,german_short-haired_pointer,french_bulldog,bouvier_des_flandres,tibetan_mastiff,english_springer,cocker_spaniel,rottweiler
0,1828590efd8ccb22737c9a7223bd8b5c,0.0000,0.0001,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,...,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0
1,1834e4243c1b87f50239cb519d7b69d1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,...,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0002,0.0000,0.0000,0.0
2,18aeddcc3ce1bb9aad5f9d1d8f4154df,0.0000,0.0049,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,...,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0
3,18b4b3e234b18b3df5fca201c2572f4f,0.0003,0.0055,0.0077,0.0012,0.0006,0.0262,0.0,0.0024,0.0073,...,0.0002,0.0001,0.002,0.0003,0.0011,0.0073,0.0000,0.1208,0.0001,0.0
4,1895f3fe0d30df6762d0e9602655d95c,0.0007,0.0001,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,...,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0
